# Manipulating data

In this chapter, you'll learn about the pyspark.sql module, which provides optimized data queries to your Spark session.

## Preparing the environment

### Importing libraries

In [1]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [2]:
from pprint import pprint

from pyspark.sql.types import (_parse_datatype_string, StructType, StructField,
                               DoubleType, IntegerType, StringType)
from pyspark.sql import SparkSession, functions as F
from pyspark.testing.utils import assertDataFrameEqual

### Connect to Spark

In [3]:
spark = SparkSession.builder.getOrCreate()

# eval DataFrame in notebooks
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

### Reading the data

In [4]:
schema_str = "year int, month int, day int, dep_time int, dep_delay int, arr_time int, " + \
             "arr_delay int, carrier string, tailnum string, flight int, origin string, " + \
             "dest string, air_time int, distance int, hour int, minute int"
customSchema = _parse_datatype_string(schema_str)
flights = spark.read.csv('data-sources/flights_small.csv', header=True, schema=schema_str)
flights.createOrReplaceTempView("flights")
flights.printSchema()
flights.limit(2)

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: integer (nullable = true)
 |-- dep_delay: integer (nullable = true)
 |-- arr_time: integer (nullable = true)
 |-- arr_delay: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)



year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
2014,12,8,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,132,954,6,58
2014,1,22,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360,2677,10,40


In [5]:
schema_str = "faa string, name string, lat double, lon double, alt int, tz int, dst string"
customSchema = _parse_datatype_string(schema_str)
airports = spark.read.schema(customSchema).csv('data-sources/airports.csv', header=True)
airports.createOrReplaceTempView("airports")
airports.printSchema()
airports.limit(2)

root
 |-- faa: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- alt: integer (nullable = true)
 |-- tz: integer (nullable = true)
 |-- dst: string (nullable = true)



faa,name,lat,lon,alt,tz,dst
04G,Lansdowne Airport,41.1304722,-80.6195833,1044,-5,A
06A,Moton Field Munic...,32.4605722,-85.6800278,264,-5,A


In [6]:
customSchema = StructType([
    StructField("tailnum", StringType()),
    StructField("year", IntegerType()),
    StructField("type", StringType()),
    StructField("manufacturer", StringType()),
    StructField("model", StringType()),
    StructField("engines", IntegerType()),
    StructField("seats", IntegerType()),
    StructField("speed", DoubleType()),
    StructField("engine", StringType())
])
planes = (spark.read.schema(customSchema)
                    .format("csv")
                    .option("header", "true")
                    .load('data-sources/planes.csv'))
planes.createOrReplaceTempView("planes")
planes.printSchema()
planes.limit(2)

root
 |-- tailnum: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- engines: integer (nullable = true)
 |-- seats: integer (nullable = true)
 |-- speed: double (nullable = true)
 |-- engine: string (nullable = true)



tailnum,year,type,manufacturer,model,engines,seats,speed,engine
N102UW,1998,Fixed wing multi ...,AIRBUS INDUSTRIE,A320-214,2,182,NULL,Turbo-fan
N103US,1999,Fixed wing multi ...,AIRBUS INDUSTRIE,A320-214,2,182,NULL,Turbo-fan


In [7]:
spark.catalog.listTables()

[Table(name='airports', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='flights', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='planes', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

## Ex. 1 - Creating columns

In this chapter, you'll learn how to use the methods defined by Spark's DataFrame class to perform common data operations.

Let's look at performing column-wise operations. In Spark you can do this using the `.withColumn()` method, which takes two arguments. First, a string with the name of your new column, and second the new column itself.

The new column must be an object of class `Column`. Creating one of these is as easy as extracting a column from your DataFrame using `df.colName`.

Updating a Spark DataFrame is somewhat different than working in pandas because the Spark DataFrame is immutable. This means that it can't be changed, and so columns can't be updated in place.

Thus, all these methods return a new DataFrame. To overwrite the original DataFrame you must reassign the returned DataFrame using the method like so:

`df = df.withColumn("newCol", df.oldCol + 1)`

The above code creates a DataFrame with the same columns as `df` plus a new column, `newCol`, where every entry is equal to the corresponding entry from `oldCol`, plus one.

To overwrite an existing column, just pass the name of the column as the first argument!

**Instructions:**
1. Use the `spark.table()` method with the argument `"flights"` to create a DataFrame containing the values of the flights table in the `.catalog`. Save it as flights.
2. Show the head of flights using `flights.show()`. Check the output: the column `air_time` contains the duration of the flight in minutes.
3. Update `flights` to include a new column called `duration_hrs`, that contains the duration of each flight in hours (you'll need to divide `air_time` by the number of minutes in an hour).

In [8]:
# Create the DataFrame flights
flights = spark.table("flights")

# Show the head
flights.show()

# Add duration_hrs
flights = flights.withColumn('duration_hrs', flights.air_time/60)
flights

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute,duration_hrs
2014,12,8,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,132,954,6,58,2.2
2014,1,22,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360,2677,10,40,6.0
2014,3,9,1443,-2,1652,2,VX,N847VA,755,SEA,SFO,111,679,14,43,1.85
2014,4,9,1705,45,1839,34,WN,N360SW,344,PDX,SJC,83,569,17,5,1.3833333333333333
2014,3,9,754,-1,1015,1,AS,N612AS,522,SEA,BUR,127,937,7,54,2.1166666666666667
2014,1,15,1037,7,1352,2,WN,N646SW,48,PDX,DEN,121,991,10,37,2.0166666666666666
2014,7,2,847,42,1041,51,WN,N422WN,1520,PDX,OAK,90,543,8,47,1.5
2014,5,12,1655,-5,1842,-18,VX,N361VA,755,SEA,SFO,98,679,16,55,1.6333333333333333
2014,4,19,1236,-4,1508,-7,AS,N309AS,490,SEA,SAN,135,1050,12,36,2.25
2014,11,19,1812,-3,2352,-4,AS,N564AS,26,SEA,ORD,198,1721,18,12,3.3


## Ex. 2 - Filtering Data

Let's take a look at the `.filter()` method. As you might suspect, this is the Spark counterpart of SQL's `WHERE` clause. The `.filter()` method takes either an expression that would follow the `WHERE` clause of a SQL expression as a string, or a Spark Column of boolean (True/False) values.

For example, the following two expressions will produce the same output:

```
flights.filter("air_time > 120").show()
flights.filter(flights.air_time > 120).show()
```

Notice that in the first case, we pass a string to `.filter()`. In SQL, we would write this filtering task as `SELECT * FROM flights WHERE air_time > 120`. Spark's `.filter()` can accept any expression that could go in the `WHERE` clause of a SQL query (in this case, `"air_time > 120"`), as long as it is passed as a string. Notice that in this case, we do not reference the name of the table in the string -- as we wouldn't in the SQL request.

In the second case, we actually pass a column of boolean values to `.filter()`. Remember that `flights.air_time > 120` returns a column of boolean values that has True in place of those records in `flights.air_time` that are over 120, and False otherwise.

**Instructions:**

1. Use the `.filter()` method to find all the flights that flew over 1000 miles two ways:
    - First, pass a SQL string to `.filter()` that checks whether the distance is greater than 1000. Save this as `long_flights1`.
    - Then pass a column of boolean values to `.filter()` that checks the same thing. Save this as `long_flights2`.
2. Use `.show()` to print heads of both DataFrames and make sure they're actually equal!

In [9]:
# Filter flights by passing a string
long_flights1 = flights.filter("distance > 1000")

# Filter flights by passing a column of boolean values
long_flights2 = flights.filter(flights.distance > 1000)

# Print the data to check they're equal
long_flights1

year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute,duration_hrs
2014,1,22,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360,2677,10,40,6.0
2014,4,19,1236,-4,1508,-7,AS,N309AS,490,SEA,SAN,135,1050,12,36,2.25
2014,11,19,1812,-3,2352,-4,AS,N564AS,26,SEA,ORD,198,1721,18,12,3.3
2014,8,3,1120,0,1415,2,AS,N305AS,656,SEA,PHX,154,1107,11,20,2.566666666666667
2014,11,12,2346,-4,217,-28,AS,N765AS,121,SEA,ANC,183,1448,23,46,3.05
2014,8,11,1017,-3,1613,-7,WN,N8634A,827,SEA,MDW,216,1733,10,17,3.6
2014,1,13,2156,-9,607,-15,AS,N597AS,24,SEA,BOS,290,2496,21,56,4.833333333333333
2014,9,26,610,-5,1523,65,US,N127UW,616,SEA,PHL,293,2378,6,10,4.883333333333334
2014,12,4,954,-6,1348,-17,HA,N395HA,29,SEA,OGG,333,2640,9,54,5.55
2014,6,4,1115,0,1346,-3,AS,N461AS,488,SEA,SAN,133,1050,11,15,2.216666666666667


In [10]:
long_flights2

year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute,duration_hrs
2014,1,22,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360,2677,10,40,6.0
2014,4,19,1236,-4,1508,-7,AS,N309AS,490,SEA,SAN,135,1050,12,36,2.25
2014,11,19,1812,-3,2352,-4,AS,N564AS,26,SEA,ORD,198,1721,18,12,3.3
2014,8,3,1120,0,1415,2,AS,N305AS,656,SEA,PHX,154,1107,11,20,2.566666666666667
2014,11,12,2346,-4,217,-28,AS,N765AS,121,SEA,ANC,183,1448,23,46,3.05
2014,8,11,1017,-3,1613,-7,WN,N8634A,827,SEA,MDW,216,1733,10,17,3.6
2014,1,13,2156,-9,607,-15,AS,N597AS,24,SEA,BOS,290,2496,21,56,4.833333333333333
2014,9,26,610,-5,1523,65,US,N127UW,616,SEA,PHL,293,2378,6,10,4.883333333333334
2014,12,4,954,-6,1348,-17,HA,N395HA,29,SEA,OGG,333,2640,9,54,5.55
2014,6,4,1115,0,1346,-3,AS,N461AS,488,SEA,SAN,133,1050,11,15,2.216666666666667


In [11]:
assertDataFrameEqual(long_flights1, long_flights2, rtol=1e-1)

## Ex. 3 - Selecting

The Spark variant of SQL's SELECT is the `.select()` method. This method takes multiple arguments - one for each column you want to select. These arguments can either be the column name as a string (one for each column) or a column object (using the `df.colName` syntax). When you pass a column object, you can perform operations like addition or subtraction on the column to change the data contained in it, much like inside `.withColumn()`.

The difference between `.select()` and `.withColumn()` methods is that `.select()` returns only the columns you specify, while `.withColumn()` returns all the columns of the DataFrame in addition to the one you defined. It's often a good idea to drop columns you don't need at the beginning of an operation so that you're not dragging around extra data as you're wrangling. In this case, you would use `.select()` and not `.withColumn()`.

**Instructions**

1. Select the columns `"tailnum"`, `"origin"`, and `"dest"` from flights by passing the column names as strings. Save this as `selected1`.
2. Select the columns `"origin"`, `"dest"`, and `"carrier"` using the `df.colName` syntax and then filter the result using both of the filters already defined for you (`filterA` and `filterB`) to only keep flights from SEA to PDX. Save this as `selected2`.

In [12]:
# Select the first set of columns
selected1 = flights.select("tailnum", "origin", "dest")
selected1.limit(5)

tailnum,origin,dest
N846VA,SEA,LAX
N559AS,SEA,HNL
N847VA,SEA,SFO
N360SW,PDX,SJC
N612AS,SEA,BUR


In [13]:
# Select the second set of columns
temp = flights.select(flights.origin, flights.dest, flights.carrier)

# Define first filter
filterA = flights.origin == "SEA"

# Define second filter
filterB = flights.dest == "PDX"

# Filter the data, first by filterA then by filterB
selected2 = temp.filter(filterA).filter(filterB)
selected2.limit(5)

origin,dest,carrier
SEA,PDX,OO
SEA,PDX,OO
SEA,PDX,OO
SEA,PDX,OO
SEA,PDX,OO


## Ex. 4 - Selecting II

Similar to SQL, you can also use the `.select()` method to perform column-wise operations. When you're selecting a column using the `df.colName` notation, you can perform any column operation and the `.select()` method will return the transformed column. For example,

> `flights.select(flights.air_time/60)`

returns a column of flight durations in hours instead of minutes. You can also use the `.alias()` method to rename a column you're selecting. So if you wanted to `.select()` the column `duration_hrs` (which isn't in your DataFrame) you could do

> `flights.select((flights.air_time/60).alias("duration_hrs"))`

The equivalent Spark DataFrame method `.selectExpr()` takes SQL expressions as a string:

> `flights.selectExpr("air_time/60 as duration_hrs")`

with the SQL as keyword being equivalent to the `.alias()` method. To select multiple columns, you can pass multiple strings.

**Instructions:**

Create a table of the average speed of each flight both ways.

1. Calculate average speed by dividing the `distance` by the `air_time` (converted to hours). Use the `.alias()` method name this column `"avg_speed"`. Save the output as the variable `avg_speed`.
2. Select the columns `"origin"`, `"dest"`, `"tailnum"`, and `avg_speed` (without quotes!). Save this as `speed1`.
Create the same table using `.selectExpr()` and a string containing a SQL expression. Save this as `speed2`.

In [14]:
# Define avg_speed
avg_speed = (flights.distance/(flights.air_time/60)).alias("avg_speed")

# Select the correct columns
speed1 = flights.select("origin", "dest", "tailnum", avg_speed)
speed1.limit(5)

origin,dest,tailnum,avg_speed
SEA,LAX,N846VA,433.6363636363636
SEA,HNL,N559AS,446.1666666666667
SEA,SFO,N847VA,367.02702702702703
PDX,SJC,N360SW,411.3253012048193
SEA,BUR,N612AS,442.6771653543307


In [15]:
# Create the same table using a SQL expression
speed2 = flights.selectExpr("origin", "dest", "tailnum", "distance/(air_time/60) as avg_speed")
speed2.limit(5)

origin,dest,tailnum,avg_speed
SEA,LAX,N846VA,433.6363636363636
SEA,HNL,N559AS,446.1666666666667
SEA,SFO,N847VA,367.02702702702703
PDX,SJC,N360SW,411.3253012048193
SEA,BUR,N612AS,442.6771653543307


## Ex. 5 - Aggregating

All of the common aggregation methods, like `.min()`, `.max()`, and `.count()` are `GroupedData` methods. These are created by calling the `.groupBy()` DataFrame method. You'll learn exactly what that means in a few exercises. For now, all you have to do to use these functions is call that method on your DataFrame. For example, to find the minimum value of a column, `col`, in a DataFrame, `df`, you could do

`df.groupBy().min("col").show()`

This creates a GroupedData object (so you can use the `.min()` method), then finds the minimum value in col, and returns it as a DataFrame.

Now you're ready to do some aggregating of your own!

**Instructions:**

1. Find the length of the shortest (in terms of `distance`) flight that left `PDX` by first `.filter()`ing and using the `.min()` method. Perform the filtering by referencing the column directly, not passing a SQL string.
2. Find the length of the longest (in terms of `time`) flight that left `SEA` by `filter()`ing and using the `.max()` method. Perform the filtering by referencing the column directly, not passing a SQL string.

In [16]:
flights.limit(2)

year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute,duration_hrs
2014,12,8,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,132,954,6,58,2.2
2014,1,22,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360,2677,10,40,6.0


In [17]:
# Find the shortest flight from PDX in terms of distance
flights.filter(flights.origin == "PDX").groupBy().min('distance').show()

+-------------+
|min(distance)|
+-------------+
|          106|
+-------------+



In [18]:
# Find the longest flight from SEA in terms of air time
flights.filter(flights.origin == "SEA").groupBy().max('air_time').show()

+-------------+
|max(air_time)|
+-------------+
|          409|
+-------------+



## Ex. 6 - Aggregating II

To get you familiar with more of the built in aggregation methods, here's a few more exercises involving the flights table!

**Instructions:**

1. Use the `.avg()` method to get the average air time of Delta Airlines flights (where the carrier column has the value `"DL"`) that left `SEA`. The place of departure is stored in the column `origin`. `show()` the result.
2. Use the `.sum()` method to get the total number of hours all planes in this dataset spent in the air by creating a column called `duration_hrs` from the column `air_time`. `show()` the result.

In [19]:
# Average duration of Delta flights
flights.filter(flights.carrier == "DL").filter(flights.origin == "SEA").groupBy().avg("air_time").show()

+------------------+
|     avg(air_time)|
+------------------+
|188.20689655172413|
+------------------+



In [20]:
# Total hours in the air
flights.withColumn("duration_hrs", flights.air_time/60).groupBy().sum("duration_hrs").show()

+------------------+
| sum(duration_hrs)|
+------------------+
|25289.600000000126|
+------------------+



## Ex. 7 - Grouping and Aggregating I

Part of what makes aggregating so powerful is the addition of groups. `PySpark` has a whole class devoted to grouped data frames: `pyspark.sql.GroupedData`, which you saw in the last two exercises.

You've learned how to create a grouped DataFrame by calling the `.groupBy()` method on a DataFrame with no arguments.

Now you'll see that when you pass the name of one or more columns in your DataFrame to the `.groupBy()` method, the aggregation methods behave like when you use a `GROUP BY` statement in a SQL query!

**Instructions:**
1. Create a DataFrame called `by_plane` that is grouped by the column `tailnum`.
2. Use the `.count()` method with no arguments to count the number of flights each plane made.
3. Create a DataFrame called `by_origin` that is grouped by the column origin.
4. Find the `.avg()` of the `air_time` column to find average duration of flights from PDX and SEA.

In [21]:
# Group by tailnum
by_plane = flights.groupBy("tailnum")

# Number of flights each plane made
by_plane.count().limit(5).show()

+-------+-----+
|tailnum|count|
+-------+-----+
| N442AS|   38|
| N102UW|    2|
| N36472|    4|
| N38451|    4|
| N73283|    4|
+-------+-----+



In [22]:
# Group by origin
by_origin = flights.groupBy("origin")

# Average duration of flights from PDX and SEA
by_origin.avg("air_time").show()

+------+------------------+
|origin|     avg(air_time)|
+------+------------------+
|   SEA| 160.4361496051259|
|   PDX|137.11543248288737|
+------+------------------+



## Ex. 8 - Grouping and Aggregating II

In addition to the `GroupedData` methods you've already seen, there is also the `.agg()` method. This method lets you pass an aggregate column expression that uses any of the aggregate functions from the `pyspark.sql.functions` submodule.

This submodule contains many useful functions for computing things like standard deviations. All the aggregation functions in this submodule take the name of a column in a `GroupedData` table.

**Instructions:**

1. Import the submodule `pyspark.sql.functions` as `F`. (Already done)
2. Create a `GroupedData` table called `by_month_dest` that's grouped by both the `month` and `dest` columns. Refer to the two columns by passing both strings as separate arguments.
3. Use the `.avg()` method on the `by_month_dest` DataFrame to get the average `dep_delay` in each month for each destination.
4. Find the standard deviation of `dep_delay` by using the `.agg()` method with the function `F.stddev()`.

In [23]:
# Group by month and dest
by_month_dest = flights.groupBy('month', 'dest')

# Average departure delay by month and destination
by_month_dest.avg('dep_delay').limit(5).show()

+-----+----+------------------+
|month|dest|    avg(dep_delay)|
+-----+----+------------------+
|    4| PHX|1.6833333333333333|
|    1| RDM|            -1.625|
|    5| ONT|3.5555555555555554|
|    7| OMA|              -6.5|
|    8| MDW|              7.45|
+-----+----+------------------+



In [24]:
# Standard deviation of departure delay
by_month_dest.agg(F.stddev('dep_delay')).limit(5).show()

+-----+----+------------------+
|month|dest| stddev(dep_delay)|
+-----+----+------------------+
|    4| PHX|15.003380033491737|
|    1| RDM| 8.830749846821778|
|    5| ONT|18.895178691342874|
|    7| OMA|2.1213203435596424|
|    8| MDW|14.467659032985843|
+-----+----+------------------+



## Ex. 9 - Joining

In `PySpark`, joins are performed using the DataFrame method `.join()`. This method takes three arguments. The first is the second DataFrame that you want to join with the first one. The second argument, `on`, is the name of the key column(s) as a string. The names of the key column(s) must be the same in each table. The third argument, `how`, specifies the kind of join to perform. In this course we'll always use the value `how="leftouter"`.

**Instructions:**

1. Examine the `airports` DataFrame by calling `.show()`. Note which key column will let you join `airports` to the `flights` table.
2. Rename the `faa` column in airports to `dest` by re-assigning the result of `airports.withColumnRenamed("faa", "dest")` to `airports`.
3. Join the `flights` with the `airports` DataFrame on the `dest` column by calling the `.join()` method on `flights`. Save the result as `flights_with_airports`.
    - The first argument should be the other DataFrame, `airports`.
    - The argument `on` should be the key column.
    - The argument `how` should be `"leftouter"`.
4. Call .show() on `flights_with_airports` to examine the data again.

In [25]:
flights.limit(2)

year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute,duration_hrs
2014,12,8,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,132,954,6,58,2.2
2014,1,22,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360,2677,10,40,6.0


In [26]:
airports.limit(2)

faa,name,lat,lon,alt,tz,dst
04G,Lansdowne Airport,41.1304722,-80.6195833,1044,-5,A
06A,Moton Field Munic...,32.4605722,-85.6800278,264,-5,A


In [27]:
# Rename the faa column
airports = airports.withColumnRenamed("faa", "dest")
airports.limit(2)

dest,name,lat,lon,alt,tz,dst
04G,Lansdowne Airport,41.1304722,-80.6195833,1044,-5,A
06A,Moton Field Munic...,32.4605722,-85.6800278,264,-5,A


In [28]:
# Join the DataFrames
flights_with_airports = flights.join(airports, on='dest', how='leftouter')

# Examine the new DataFrame
flights_with_airports.limit(5)

dest,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,air_time,distance,hour,minute,duration_hrs,name,lat,lon,alt,tz,dst
LAX,2014,12,8,658,-7,935,-5,VX,N846VA,1780,SEA,132,954,6,58,2.2,Los Angeles Intl,33.942536,-118.408075,126,-8,A
HNL,2014,1,22,1040,5,1505,5,AS,N559AS,851,SEA,360,2677,10,40,6.0,Honolulu Intl,21.318681,-157.922428,13,-10,N
SFO,2014,3,9,1443,-2,1652,2,VX,N847VA,755,SEA,111,679,14,43,1.85,San Francisco Intl,37.618972,-122.374889,13,-8,A
SJC,2014,4,9,1705,45,1839,34,WN,N360SW,344,PDX,83,569,17,5,1.3833333333333333,Norman Y Mineta S...,37.3626,-121.929022,62,-8,A
BUR,2014,3,9,754,-1,1015,1,AS,N612AS,522,SEA,127,937,7,54,2.1166666666666667,Bob Hope,34.200667,-118.358667,778,-8,A


## Close

In [29]:
spark.stop()